In [ ]:
from transformers import MBartForConditionalGeneration
import torch
from transformers import AutoModel, AutoTokenizer
import time 
from tqdm import tqdm
bartpho = AutoModel.from_pretrained("vinai/bartpho-syllable")
bartpho = MBartForConditionalGeneration.from_pretrained("vinai/bartpho-syllable")
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
pipeline = pipeline(
    task="fill-mask",
    model="vinai/phobert-base",
    torch_dtype=torch.float16,
    device=0
)
def remove_special_characters(text):
    import re
    # Remove special characters and digits, keep only Vietnamese characters and spaces
    text = re.sub(r'[^a-zA-ZÀ-ỹ\s]', '', text)
    return text.strip()
def topk_predictions(text, top_k=5):
    # input_ids = tokenizer([text], return_tensors="pt")["input_ids"]
    # logits = bartpho(input_ids).logits
    # masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()

    # probs = logits[0, masked_index].softmax(dim=0)
    # values, predictions = probs.topk(top_k)
    
    # return tokenizer.decode(predictions).split()
    
    output = pipeline(text, top_k=top_k)
    mask_index = text.index('<mask>')
    # Extract the predictions from the output
    predictions = []
    for item in output:
        if 'token_str' in item:
            predictions.append(item['token_str'])
    
    predictions = [remove_special_characters(item['token_str']) for item in output]

def sentence_prediction(text, top_k=30):
    error_indices = []
    for i in range(len(text.split())):
        original_word
        new_list = text.split()
        # replace the i-th word with a mask token
        # clean_word = remove_special_characters(new_list[i])
        clean_word = new_list[i]
        new_list[i] = "<mask>"
        masked_text = " ".join(new_list)
        start_time = time.time()
        predictions = topk_predictions(masked_text, top_k=top_k)
        # for j in range(len(predictions)):
        #     predictions[j] = remove_special_characters(predictions[j])
        end_time = time.time()
        if clean_word not in predictions:
            error_indices.append((i, clean_word, predictions[0]))
            print(f"Masked Text: {masked_text}")
            print(f"->Top {top_k} Predictions: {predictions}")
            
        # print(f"Masked Text: {masked_text}")
    return error_indices

def paragraph_prediction(text, top_k=30):
    sentences = text.split('. ')
    all_errors = []
    for sentence in sentences:
        errors = sentence_prediction(sentence, top_k=top_k)
        if errors:
            all_errors.append((sentence, errors))
            
    return all_errors

In [55]:

TXT = "Trung tâm Dự báo khí tượng thủy văn quốc gia cho biết lúc 7h hôm nay, áp thấp nhiệt đới mạnh 61 km/h, cấp 6-7, "
TXT = "Theo thầy Đỗ Đình Đảo, Hiệu trưởng, những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây."
paragraph_prediction(TXT, top_k=100)

Masked Text: Theo thầy Đỗ Đình <mask> Hiệu trưởng, những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây.
->Top 100 Predictions: ['Th', 'T', 'C', 'Hùng', 'Ph', 'Ho', 'Trung', 'Phụ', 'Kh', 'Qu', 'Tuấn', 'L', 'Đức', 'Vinh', 'Khánh', 'Tr', 'Thuận', 'Hồng', 'H', 'Tiến', 'Việt', 'Sơn', 'Hà', 'Giao', 'Thắng', 'Thu', 'Ngh', 'Khoa', 'Hiệp', 'Khả', 'Bình', 'Diễn', 'Tu', 'Ki', 'Châu', 'Quý', 'S', 'Tài', 'Hu', 'Phi', 'Bá', 'D', 'Phong', 'Viên', 'Đ', 'Ng', 'Tân', 'Tâm', 'Hợp', 'Ngọc', 'Ch', 'Phúc', 'Thái', 'Á', 'Hưng', 'Phú', 'Thanh', 'Dũng', 'Â', 'Duy', 'Toàn', 'Quy', 'Liên', 'Dung', 'Phương', 'Long', 'Thảo', 'Nguyên', 'Chương', 'Hoàn', 'X', 'Trọng', 'Thị', 'M', 'Bí', 'Thi', 'Di', 'Hi', 'Thành', 'Độ', 'Minh', 'Tho', 'Loan', 'Văn', 'Tru', 'Quang', 'Hiển', 'V', 'Thân', 'Lợi', 'Bảo', 'B', 'Tuy', 'Anh', 'Tha', 'K', 'San', 'Hải', 'Chính']
Masked Text: Theo thầy Đỗ Đình Đảo, Hiệu <mask> những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây.


[('Theo thầy Đỗ Đình Đảo, Hiệu trưởng, những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây.',
  [(4, 'Đảo,', 'Th'),
   (6, 'trưởng,', 'trưởng'),
   (18, 'vào', 'và'),
   (20, 'tựu', 'khai'),
   (22, '20/8', '20,'),
   (23, 'tới', 'tới./2010,'),
   (24, 'đây.', '.')])]

In [52]:

#BARTpho-word
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word")


In [53]:
TXT = 'Trung tâm Dự báo khí tượng thủy văn quốc gia cho biết lúc 7h hôm nay, áp thấp nhiệt đới mạnh 61 km/h, cấp 6-7,'  
input_ids = word_tokenizer(TXT, return_tensors='pt')['input_ids']
features = bartpho_word(input_ids).logits


AttributeError: 'Seq2SeqModelOutput' object has no attribute 'logits'

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Tôi là <mask> trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])



AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [59]:
with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

# Extract the logits


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'score': 0.51025390625,
  'token': 3716,
  'token_str': 'Hiệu_trưởng',
  'sequence': 'Tôi là Hiệu_trưởng trường đại_học Công_nghệ .'},
 {'score': 0.2998046875,
  'token': 4856,
  'token_str': 'hiệu_trưởng',
  'sequence': 'Tôi là hiệu_trưởng trường đại_học Công_nghệ .'},
 {'score': 0.039947509765625,
  'token': 3466,
  'token_str': 'giảng_viên',
  'sequence': 'Tôi là giảng_viên trường đại_học Công_nghệ .'},
 {'score': 0.03753662109375,
  'token': 250,
  'token_str': 'Chủ_tịch',
  'sequence': 'Tôi là Chủ_tịch trường đại_học Công_nghệ .'},
 {'score': 0.0199432373046875,
  'token': 649,
  'token_str': 'sinh_viên',
  'sequence': 'Tôi là sinh_viên trường đại_học Công_nghệ .'},
 {'score': 0.00891876220703125,
  'token': 1623,
  'token_str': 'chủ_tịch',
  'sequence': 'Tôi là chủ_tịch trường đại_học Công_nghệ .'},
 {'score': 0.0084381103515625,
  'token': 693,
  'token_str': 'Giám_đốc',
  'sequence': 'Tôi là Giám_đốc trường đại_học Công_nghệ .'},
 {'score': 0.007450103759765625,
  'token': 33

In [67]:
from pyvi import ViTokenizer, ViPosTagger

ViTokenizer.tokenize(u"Trường đại học bách khoa hà nội")

'Trường đại_học bách_khoa hà_nội'

In [96]:
# replace uỷ with ủy
dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }


In [ ]:
TXT = "Trung tâm Dự báo Khí tượng Thủy văn quốc gia cho biết lúc 7h hôm na, áp thấp nhiệt đới mạnh 61 km/h, cấp 6-7,"
# replace uỷ with ủy
dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }
from pyvi import ViTokenizer, ViPosTagger

ViTokenizer.tokenize(u"Trường đại học bách khoa hà nội")
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
def process_text(text):
    for key, value in dict_map.items():
        text = text.replace(key, value)
    return text
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
def sentence_prediction(text, top_k=30):
    text = ViTokenizer.tokenize(TXT)    
    
    error_indices = []
    for i in range(len(text.split())):
        text = process_text(text)
        new_list = text.split()
        original_word = new_list[i]
        if is_number(original_word):
            continue
        
        new_list[i] = tokenizer.mask_token
        masked_text = " ".join(new_list)
        start_time = time.time()
        predictions = pipeline(masked_text, top_k=100)
        topk = []
        for item in predictions:
            if 'token_str' in item:
                topk.append(item['token_str'])
        end_time = time.time()
        if original_word not in topk:
            print(f"Original Word: {original_word}")
            print(f"Masked Text: {masked_text}")
            print(f"->Top 10 Predictions: {topk}")
            error_indices.append((i, original_word, topk[0]))
        # print(f"Masked Text: {masked_text}")
    return error_indices

   

In [104]:
print(sentence_prediction(TXT, top_k=100))

Original Word: na
Masked Text: Trung_tâm Dự_báo Khí_tượng Thuỷ_văn quốc_gia cho biết lúc 7h hôm <mask> , áp_thấp nhiệt_đới mạnh 61 km / h , cấp 6 - 7 ,
->Top 10 Predictions: ['nay', 'sau', 'trước', 'đó', '28/7', '23/7', '18/7', '19/7', '17/7', '10/10', 'ấy', '23/8', '22/7', '9/10', '5/5', '11/7', '19/8', '15/9', '5/8', '22/8', '23/6', '4/7', '5/10', '5/4', '26/7', '18/8', '24/2', '18/5', '8/8', 'này', '20/7', '29/8', 'qua', '25/7', '21/6', '27/7', '30/7', '3/8', '10/5', '7/10', '18/11', '16/7', '21/7', '1/6', '25/5', '20/11', '24/8', '11/6', '21/5', '15/5', '8/3', '27/5', '12/7', '9/5', '25/8', '15/7', '21/11', '18/6', '13/7', '15/10', '5/6', '7/12', '21/10', '16/8', '15/6', '6/8', '12/4', '19/10', '10/8', '22/10', '18/10', '17/8', '29/7', '27/6', '31/7', '3/5', '9/6', '13/10', '17/6', '4/6', '15/8', '4/11', '8/10', '10/11', '20/4', '21/4', '22/6', '11/10', '31/8', '12/10', '3/6', '19/11', '14/7', '29/9', '12/5', '11/3', '22/2', '4/10', '1/7', '17/5']
[(10, 'na', 'nay')]
